In [ ]:
import os
import json
import fiona
import networkx as nx
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from shapely import wkt
from gnn_package import data_utils
from gnn_package import preprocessing
from private_uoapi import LightsailWrapper, LSAuth, LSConfig

import gnn_package.src.preprocessing.graph_utils as ppgu
import gnn_package.src.preprocessing.graph_manipulation as ppgm
import gnn_package.src.preprocessing.graph_analysis as ppga
import gnn_package.src.preprocessing.graph_computation as ppgc
import gnn_package.src.preprocessing.graph_visualization as ppgv

In [ ]:
# Example usage
place_name = "Newcastle upon Tyne, UK"  # Replace with your area of interest

# Get the network
network_gdf = preprocessing.graph_utils.get_street_network_gdfs(place_name)

In [ ]:
sensor_id_map = ppgu.get_senso_name_id_map()

In [ ]:
sensors_gdf = ppgu.read_or_create_sensor_nodes()

In [ ]:
# Get the bounding box
bbox_transformed = ppgu.get_bbox_transformed()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
network_gdf.plot(ax=ax, color="black", alpha=0.5)
sensors_gdf.plot(ax=ax, color="red", markersize=10)

In [ ]:
# First verify the CRS of both dataframes
print("Roads CRS:", network_gdf.crs)
print("Private sensor nodes CRS:", sensors_gdf.crs)

# Create the plot with a larger figure size
fig, ax = plt.subplots(figsize=(12, 12))

# Plot highways and paths
network_gdf.plot(ax=ax, color="grey", linewidth=0.5)

sensors_gdf.plot(
    ax=ax,
    color="red",
    markersize=20,
    alpha=0.6,  # Add some transparency
    zorder=2,  # Ensure points are plotted on top
)

# Set the plot bounds based on the highways extent - there are some sensors that are a long way from the roads
bounds = network_gdf.total_bounds
ax.set_xlim([bounds[0], bounds[2]])
ax.set_ylim([bounds[1], bounds[3]])

# Add gridlines
ax.grid(True)

# Add title
plt.title("OSM - Paths")

# Show the plot
plt.tight_layout()
plt.show()

# Print some diagnostic information
print("\nHighways bounds:", network_gdf.total_bounds)
print("Private sensor nodes bounds:", sensors_gdf.total_bounds)